# Step 1: Install Required Libraries

In [2]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Step 2: Load and Preprocess the Data

In [3]:
import pandas as pd

# Load the dataset
data = pd.read_excel("com_balanced-3.xlsx")

# Remove any rows with missing values
data.dropna(inplace=True)

# Convert the sentiment labels to integers
data['label'] = data['label'].astype(int)

# Print the first 5 rows of the data
print(data.head())


   label                                           comments
0      1  жақсы бір өзіне сай азамат таппадынба? Шайтан ...
1      2  Ринат биликтеглердин созин соилеп калыпсынгои ...
2      2  Ыгаими мен сыгаи мусылман аты садака кетсин ак...
3      1                               Позор еркекте ес жоқ
4      0  Атырауға қыдырып емес жұмыс жасауға келгендер ...


# Step 3: Split the Data into Train and Test Sets

### We need to split the data into a training set and a test set. We use 80% of the data for training and 20% for testing.

In [4]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Print the shapes of the train and test sets
print("Train set shape:", train_data.shape)
print("Test set shape:", test_data.shape)



Train set shape: (6715, 2)
Test set shape: (1679, 2)


# Step 4: Load the Pre-trained BERT Model
### Next, we need to load the pre-trained BERT model from Hugging Face. We use the "bert-base-multilingual-cased" model, which supports multiple languages including Kazakh and Russian. 

In [5]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

# Step 5: Tokenize the Text Data

In [6]:
import torch

# Tokenize the train and test data
train_tokens = tokenizer.batch_encode_plus(
    train_data['comments'].tolist(),
    max_length = 256,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)

test_tokens = tokenizer.batch_encode_plus(
    test_data['comments'].tolist(),
    max_length = 256,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)

# Convert the tokenized data into PyTorch tensors
train_seq = torch.tensor(train_tokens['input_ids'])
train_mask = torch.tensor(train_tokens['attention_mask'])
train_y = torch.tensor(train_data['label'].tolist())

test_seq = torch.tensor(test_tokens['input_ids'])
test_mask = torch.tensor(test_tokens['attention_mask'])
test_y = torch.tensor(test_data['label'].tolist())


# Step 6: Create a DataLoader

In [7]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Create a DataLoader for the train data
train_dataset = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=4)

# Create a DataLoader for the test data
test_dataset = TensorDataset(test_seq, test_mask, test_y)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=4)

# Step 7: Fine-tune the BERT Model

In [8]:
from transformers import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model.to(device)

# Set the optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=2e-5)

# Set the number of training epochs
epochs = 10
steps_per_epoch = len(train_dataloader)

# Initialize lists to store training history
train_loss_history = []
train_accuracy_history = []

# Fine-tune the BERT model
for epoch in range(epochs):
    model.train()
    total_loss = 0
    total_correct = 0
    total_examples = 0
    
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}', unit='batch')
    
    # Iterate over the train data in batches
    for step, batch in enumerate(progress_bar):
        # Move the batch to the device
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(**inputs)
        loss = outputs.loss
        logits = outputs.logits
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Calculate training accuracy
        predictions = logits.argmax(dim=1)
        correct = (predictions == inputs['labels']).sum().item()
        total_correct += correct
        total_examples += inputs['labels'].shape[0]
        
        # Print training loss and accuracy per 5000 steps
        if (step + 1) % 5000 == 0:
            avg_loss = total_loss / (step + 1)
            epoch_accuracy = total_correct / total_examples
            print(f"Training Loss per 5000 steps: {avg_loss:.4f}")
            print(f"Training Accuracy per 5000 steps: {epoch_accuracy * 100:.2f}")
            
            # Store training loss and accuracy for visualization
            train_loss_history.append(avg_loss)
            train_accuracy_history.append(epoch_accuracy)
    
    # Calculate average loss and accuracy for the epoch
    avg_loss = total_loss / steps_per_epoch
    epoch_accuracy = total_correct / total_examples
    
    # Calculate accuracy on the entire training set
    model.eval()
    train_predictions = []
    train_labels = []
    
    with torch.no_grad():
        for batch in train_dataloader:
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids': batch[0],
                      'attention_mask': batch[1],
                      'labels': batch[2]}
            
            outputs = model(**inputs)
            logits = outputs.logits
            predictions = logits.argmax(dim=1)
            
            train_predictions.extend(predictions.tolist())
            train_labels.extend(inputs['labels'].tolist())
    
    train_accuracy = accuracy_score(train_labels, train_predictions)
    
    print(f'Epoch {epoch+1} - Training Loss: {avg_loss:.4f} - Training Accuracy: {train_accuracy:.4f} - Total Accuracy: {epoch_accuracy:.4f}')

C:\Users\ASUS ROG\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1: 100%|████████████████████████████████████████████████████████████████| 1679/1679 [3:27:06<00:00,  7.40s/batch]


Epoch 1 - Training Loss: 0.9492 - Training Accuracy: 0.6618 - Total Accuracy: 0.5367


Epoch 2: 100%|████████████████████████████████████████████████████████████████| 1679/1679 [3:17:51<00:00,  7.07s/batch]


Epoch 2 - Training Loss: 0.6641 - Training Accuracy: 0.8850 - Total Accuracy: 0.7361


Epoch 3: 100%|████████████████████████████████████████████████████████████████| 1679/1679 [3:18:37<00:00,  7.10s/batch]


...

Epoch 8 - Training Loss: 0.1199 - Training Accuracy: 0.9759 - Total Accuracy: 0.9619


Epoch 9: 100%|████████████████████████████████████████████████████████████████| 1679/1679 [3:21:00<00:00,  7.18s/batch]


Epoch 9 - Training Loss: 0.0981 - Training Accuracy: 0.9823 - Total Accuracy: 0.9707


Epoch 10: 100%|███████████████████████████████████████████████████████████████| 1679/1679 [2:39:20<00:00,  5.69s/batch]


Epoch 10 - Training Loss: 0.0976 - Training Accuracy: 0.9796 - Total Accuracy: 0.9692


# Step 8: Evaluate the Model

### After fine-tuning the model, we evaluate its performance on the test data

In [9]:
from sklearn.metrics import accuracy_score, classification_report

model.eval()
predictions = []
true_labels = []

# Disable gradient calculation
with torch.no_grad():
    # Iterate over the test data in batches
    for batch in test_dataloader:
        # Move the batch to the device
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].detach().cpu().numpy()
        
        # Store predictions and true labels
        predictions.extend(logits.argmax(axis=1))
        true_labels.extend(label_ids)
        
# Calculate accuracy and print the classification report
accuracy = accuracy_score(true_labels, predictions)
classification_rep = classification_report(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')


Accuracy: 0.7748659916617034
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.62      0.73       607
           1       0.71      0.89      0.79       520
           2       0.77      0.84      0.80       552

    accuracy                           0.77      1679
   macro avg       0.79      0.78      0.77      1679
weighted avg       0.79      0.77      0.77      1679



In [19]:
import torch
from transformers import BertTokenizer

def predict_sentiment(text):
    # Tokenize the input text
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    encoded_inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_inputs['input_ids']
    attention_mask = encoded_inputs['attention_mask']

    # Move the input tensors to the device
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Apply softmax activation
        probabilities = torch.softmax(logits, dim=1)

        # Get the predicted label
        predicted_label = torch.argmax(probabilities, dim=1).item()

    # Define the sentiment labels
    sentiment_labels = ['Neutral', 'Insult', 'Sarkazm']

    return sentiment_labels[predicted_label]


In [21]:
text1 = "Мына психологка психолог керек кой"
text2 = "Шаңырағы биік, керегесі кең болсын. Сол үйде өнсін, өссін."
text3 = "Мектепті екіге бітіргендер полиция үшке бітіргендер депутат болатын заман"
text4 = "ешекке мініп жоғал"
text4 = "Өз ұлтымызды құрметтеп өз тілімізде сөйлейік, өзге ұлттарға өз тілімізді үйретейік."
text5 = "Роботтар жақсы олардан"
text6 = "Каспий редпен таксиге отырып жетісеміз десиш"
text7 = "Замшивые туфли гой бұлардікі😂"
text8 = "Түптің түбінде Қазақстанға да санкция салғызып алмаса жарар еді."


print(predict_sentiment(text1))
print(predict_sentiment(text2))
print(predict_sentiment(text3))
print(predict_sentiment(text4))
print(predict_sentiment(text5))
print(predict_sentiment(text6))
print(predict_sentiment(text7))
print(predict_sentiment(text8))

Sarkazm
Sarkazm
Insult
Sarkazm
Sarkazm
Sarkazm
Sarkazm
Sarkazm
